In [1]:
import numpy as np
import glob
from os import path
import re

In [2]:
# process results into csv
results = {}
pattern = re.compile(r"(?P<func>(entropy_score|least_confidence|random_score))_(?P<seed_size>\d+)_(?P<active_size>\d+)")
for file in glob.glob("results_ablation/**"):
    match = re.search(pattern, file)
    print(match.group("func"), match.group("seed_size"), match.group("active_size"))
    

NameError: name 're' is not defined